In [1]:
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import MonthEnd
import numpy as np
from statistics import mean
from math import floor

In [2]:
#Importing data for NIFTY Tickers

prices = pd.read_csv("Data.csv").set_index('Date')
prices.index = pd.to_datetime(prices.index)

In [3]:
def Momentum_Profit(FormDate,Period):

  prices_end = prices.loc[FormDate-MonthEnd(1)]
  prices_begin = prices.loc[FormDate-MonthEnd(12)]

  returns = prices_end/prices_begin-1
  returns = returns.reset_index().dropna()
  returns['quantiles'] = pd.qcut(returns.iloc[:,1],10,labels=False)

  winners = returns[returns.quantiles==9]
  winnerret = prices.loc[FormDate+MonthEnd(Period),winners['index']]/prices.loc[FormDate,winners['index']]
  return np.mean(winnerret)

In [4]:
#backtesting for J=12

J=12

profits,dates,holding =[],[],[]
for i in (1,3,6):
  for j in prices.index[J:-i]: 
    profits.append(Momentum_Profit(j,i))
    dates.append(j+MonthEnd(i))
    holding.append(i)

In [23]:
frame = pd.DataFrame({'Momentum_Profits':profits,'Holding_Periods':holding},index=dates)
final = frame.groupby('Holding_Periods').mean()

for i in range(len(final)):
  print("For Holding period ",str(final.index[i])," the average Momentum Profit is = ",str(final.iat[i,0]))

For Holding period  1  the average Momentum Profit is =  1.0207364887018013
For Holding period  3  the average Momentum Profit is =  1.0659668556544182
For Holding period  6  the average Momentum Profit is =  1.1430777380190829
